### Installing required libraries

In [1]:
!pip install transformers diffusers torch pillow gradio

  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached uvicorn-0.34.2-py3-none-any.whl.metadata (6.5 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/3.6 MB ? eta -:--:--
   -- ------------------------------------- 0.3/3.6 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/3.6 MB 1.9 MB/s eta 0:00:02
   ----------- ---------------------------- 1.

### Importing required libraries

In [2]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler, LMSDiscreteScheduler, \
    PNDMScheduler, DPMSolverMultistepScheduler, DDIMScheduler
import torch
import gradio as gr
from PIL import Image

c:\Users\nikhil kumar\.conda\envs\langchai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip install accelerate
!pip install hf_xet


   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.7 MB 1.3 MB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.7 MB 1.6 MB/s eta 0:00:01
   ---------------------------------- ----- 2.4/2.7 MB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 2.4 MB/s eta 0:00:00


### Load model pipeline

In [ ]:
model_id = "stabilityai/stable-diffusion-2-1"
sd_pipeline = StableDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16
)
sd_pipeline.to("cpu")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Loading pipeline components...: 100%|██████████| 6/6 [00:42<00:00,  7.11s/it]


AssertionError: Torch not compiled with CUDA enabled

### Defining multiple scheduler algorithms

In [ ]:
schedulers = {
    "Euler Discrete": EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "LMS Discrete": LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "PNDM": PNDMScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "DPM Solver Multistep": DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler"),
    "DDIM": DDIMScheduler.from_pretrained(model_id, subfolder="scheduler"),
}

### Image generating function

In [ ]:
def generate_image(
    prompt,
    negative_prompt="",
    num_inference_steps=50,
    guidance_scale=7.5,
    height=512,
    width=512,
    batch_size=1,
    seed=None,
    scheduler="Euler Discrete",
):
    # Set the selected scheduler
    sd_pipeline.scheduler = schedulers.get(scheduler, sd_pipeline.scheduler)

    # Set seed for reproducibility
    generator = torch.manual_seed(seed) if seed else None

    # Generate images
    images = sd_pipeline(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        height=height,
        width=width,
        num_images_per_prompt=batch_size,
        generator=generator
    ).images

    return images  # Always return a list, even if batch_size = 1

### Gradio interface

In [ ]:
interface = gr.Interface(
    fn=generate_image,
    inputs=[
        gr.Textbox(label="Prompt", info="Describe the image you want to generate."),
        gr.Textbox(label="Negative Prompt", info="Specify what to avoid in the image."),
        gr.Slider(step=1, minimum=1, maximum=100, value=50, label="Num Inference Steps",
                  info="Controls the number of denoising steps."),
        gr.Slider(step=0.1, minimum=1, maximum=20, value=7.5, label="Guidance Scale",
                  info="Higher values guide the image closer to the text prompt."),
        gr.Slider(step=64, minimum=256, maximum=1024, value=512, label="Height",
                  info="Set the height of the generated image."),
        gr.Slider(step=64, minimum=256, maximum=1024, value=512, label="Width",
                  info="Set the width of the generated image."),
        gr.Slider(step=1, minimum=1, maximum=4, value=1, label="Batch Size",
                  info="Generate multiple images at once (1-4)."),
        gr.Number(label="Seed", info="Set a seed for reproducibility (Optional)."),
        gr.Dropdown(
            choices=["Euler Discrete", "LMS Discrete", "PNDM", "DPM Solver Multistep", "DDIM"],
            value="Euler Discrete",
            label="Scheduler Algorithm",
            info="Choose the denoising algorithm used for image generation."
        ),
    ],
    outputs=gr.Gallery(label="Generated Images"),
    title="Gen AI Text-to-Image Generator using Stable Diffusion",
    description="Generate AI images from text with customizable settings(parameters).",
)

# Launch Gradio App
interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://43191ee283ba97295e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://43191ee283ba97295e.gradio.live
